In [52]:
#This code is based of that in https://github.com/aniruddhraghu/sepsisrl/blob/master/continuous/sarsa_physician.ipynb

In [29]:
import tensorflow as tf
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
try:
    import cPickle as pickle
except ImportError:
    import _pickle as pickle
import copy
import matplotlib.pyplot as plt

In [30]:
state_features = ['gender', 'age', 're_admission',
     'Weight_kg', 'GCS', 'HR', 'SysBP', 'MeanBP', 'DiaBP',
       'RR', 'SpO2', 'Temp_C', 'FiO2_1', 'Potassium', 'Sodium', 'Chloride',
       'Glucose', 'BUN', 'Creatinine', 'Magnesium', 'Calcium', 'SGOT', 'SGPT',
       'Total_bili', 'Hb', 'WBC_count', 'Platelets_count', 'PTT', 'PT', 'INR',
       'Arterial_pH', 'paO2', 'paCO2', 'Arterial_BE', 'Arterial_lactate',
       'HCO3', 'mechvent', 'Shock_Index', 'PaO2_FiO2', 'input_total', 'input_4hourly', 'output_total',
       'output_4hourly', 'cumulated_balance', 'SOFA', 'SIRS', 'reward',
       'vaso_input']

In [31]:
df = pd.read_csv('../Fresh_start/rl_train_set_scaled_rewarded_abchange.csv')

In [32]:
# df = df.drop("elixhauser", axis=1)
# df = df.drop("died_in_hosp", axis=1)
# df = df.drop("delay_end_of_record_and_discharge_or_death", axis=1)
# df = df.drop("died_within_48h_of_out_time", axis=1)
# df = df.drop("Ionised_Ca", axis=1)
# df = df.drop("CO2_mEqL", axis=1)
# df = df.drop("Albumin", axis=1)
# df = df.drop("charttime", axis=1)
# list_of_nan = []
# for i, row in df.iterrows():
#     if pd.isnull(row["bloc"]):
#             #print(i)
#         list_of_nan.append(i)
#             #print(list_of_nan)
# df = df.drop(list_of_nan)
# df = df.fillna(0)
# df.index = range(len(df))

In [33]:
val_df = pd.read_csv('../Fresh_start/rl_val_set_scaled_rewarded_abchange.csv')

In [34]:
test_df = pd.read_csv('../Fresh_start/rl_test_set_scaled_rewarded_abchange.csv')

In [35]:
preg_df = pd.read_csv('../Fresh_start/rl_pregnant_set_scaled_rewarded_abchange.csv')
control_1_df = pd.read_csv('../Fresh_start/rl_control_1_set_scaled_rewarded_abchange.csv')
control_2_df = pd.read_csv('../Fresh_start/rl_control_2_set_scaled_rewarded_abchange.csv')
old_df = pd.read_csv('../Fresh_start/rl_old_set_scaled_rewarded_abchange.csv')

In [36]:
# val_df = val_df.drop("elixhauser", axis=1)
# val_df = val_df.drop("died_in_hosp", axis=1)
# val_df = val_df.drop("delay_end_of_record_and_discharge_or_death", axis=1)
# val_df = val_df.drop("died_within_48h_of_out_time", axis=1)
# val_df = val_df.drop("Ionised_Ca", axis=1)
# val_df = val_df.drop("CO2_mEqL", axis=1)
# val_df = val_df.drop("Albumin", axis=1)
# val_df = val_df.drop("charttime", axis=1)
# list_of_nan = []
# for i, row in val_df.iterrows():
#     if pd.isnull(row["bloc"]):
#             #print(i)
#         list_of_nan.append(i)
#             #print(list_of_nan)
# val_df = val_df.drop(list_of_nan)
# val_df = val_df.fillna(0)
# val_df.index = range(len(val_df))
# ##################################################################
# test_df = test_df.drop("elixhauser", axis=1)
# test_df = test_df.drop("died_in_hosp", axis=1)
# test_df = test_df.drop("delay_end_of_record_and_discharge_or_death", axis=1)
# test_df = test_df.drop("died_within_48h_of_out_time", axis=1)
# test_df = test_df.drop("Ionised_Ca", axis=1)
# test_df = test_df.drop("CO2_mEqL", axis=1)
# test_df = test_df.drop("Albumin", axis=1)
# test_df = test_df.drop("charttime", axis=1)
# list_of_nan = []
# for i, row in test_df.iterrows():
#     if pd.isnull(row["bloc"]):
#             #print(i)
#         list_of_nan.append(i)
#             #print(list_of_nan)
# test_df = test_df.drop(list_of_nan)
# test_df = test_df.fillna(0)
# test_df.index = range(len(test_df))

In [37]:
# df_preg = pd.read_csv('../Created Data/rl_pregnant_data_final_cont.csv')
# df_preg = df_preg.drop("elixhauser", axis=1)
# df_preg = df_preg.drop("died_in_hosp", axis=1)
# df_preg = df_preg.drop("delay_end_of_record_and_discharge_or_death", axis=1)
# df_preg = df_preg.drop("died_within_48h_of_out_time", axis=1)
# df_preg = df_preg.drop("Ionised_Ca", axis=1)
# df_preg = df_preg.drop("CO2_mEqL", axis=1)
# df_preg = df_preg.drop("Albumin", axis=1)
# df_preg = df_preg.drop("charttime", axis=1)
# list_of_nan = []
# for i, row in df_preg.iterrows():
#     if pd.isnull(row["bloc"]):
#             #print(i)
#         list_of_nan.append(i)
#             #print(list_of_nan)
# df_preg = df_preg.drop(list_of_nan)
# df_preg = df_preg.fillna(0)
# df_preg.index = range(len(df_preg))
# ########################################################################################################################
# df_cont = pd.read_csv('../Created Data/rl_control_data_final_cont.csv')
# df_cont = df_cont.drop("elixhauser", axis=1)
# df_cont = df_cont.drop("died_in_hosp", axis=1)
# df_cont = df_cont.drop("delay_end_of_record_and_discharge_or_death", axis=1)
# df_cont = df_cont.drop("died_within_48h_of_out_time", axis=1)
# df_cont = df_cont.drop("Ionised_Ca", axis=1)
# df_cont = df_cont.drop("CO2_mEqL", axis=1)
# df_cont = df_cont.drop("Albumin", axis=1)
# df_cont = df_cont.drop("charttime", axis=1)
# list_of_nan = []
# for i, row in df_cont.iterrows():
#     if pd.isnull(row["bloc"]):
#             #print(i)
#         list_of_nan.append(i)
#             #print(list_of_nan)
# df_cont = df_cont.drop(list_of_nan)
# df_cont = df_cont.fillna(0)
# df_cont.index = range(len(df_cont))
# #########################################################################################################################
# df_cont_sec = pd.read_csv('../Created Data/rl_control_sec_data_final_cont.csv')
# df_cont_sec = df_cont_sec.drop("elixhauser", axis=1)
# df_cont_sec = df_cont_sec.drop("died_in_hosp", axis=1)
# df_cont_sec = df_cont_sec.drop("delay_end_of_record_and_discharge_or_death", axis=1)
# df_cont_sec = df_cont_sec.drop("died_within_48h_of_out_time", axis=1)
# df_cont_sec = df_cont_sec.drop("Ionised_Ca", axis=1)
# df_cont_sec = df_cont_sec.drop("CO2_mEqL", axis=1)
# df_cont_sec = df_cont_sec.drop("Albumin", axis=1)
# df_cont_sec = df_cont_sec.drop("charttime", axis=1)
# list_of_nan = []
# for i, row in df_cont_sec.iterrows():
#     if pd.isnull(row["bloc"]):
#             #print(i)
#         list_of_nan.append(i)
#             #print(list_of_nan)
# df_cont_sec = df_cont_sec.drop(list_of_nan)
# df_cont_sec = df_cont_sec.fillna(0)
# df_cont_sec.index = range(len(df_cont_sec))

In [38]:
# df_old = pd.read_csv('../Created Data/rl_old_data_final_cont.csv')
# df_old = df_old.drop("elixhauser", axis=1)
# df_old = df_old.drop("died_in_hosp", axis=1)
# df_old = df_old.drop("delay_end_of_record_and_discharge_or_death", axis=1)
# df_old = df_old.drop("died_within_48h_of_out_time", axis=1)
# df_old = df_old.drop("Ionised_Ca", axis=1)
# df_old = df_old.drop("CO2_mEqL", axis=1)
# df_old = df_old.drop("Albumin", axis=1)
# df_old = df_old.drop("charttime", axis=1)
# list_of_nan = []
# for i, row in df_old.iterrows():
#     if pd.isnull(row["bloc"]):
#             #print(i)
#         list_of_nan.append(i)
#             #print(list_of_nan)
# df_old = df_old.drop(list_of_nan)
# df_old = df_old.fillna(0)
# df_old.index = range(len(df_old))

In [39]:
REWARD_THRESHOLD = 20
reg_lambda = 5

In [40]:
# PER important weights and params
per_flag = True
beta_start = 0.9
df['prob'] = abs(df['reward'])
temp = 1.0/df['prob']
#temp[temp == float('Inf')] = 1.0
df['imp_weight'] = pow((1.0/len(df) * temp), beta_start)

In [41]:
hidden_1_size = 128
hidden_2_size = 128
#  Q-network uses Leaky ReLU activation
class Qnetwork():
    def __init__(self):
        self.phase = tf.placeholder(tf.bool)

        self.num_actions = 25

        self.input_size = len(state_features)

        self.state = tf.placeholder(tf.float32, shape=[None, self.input_size],name="input_state")

        self.fc_1 = tf.contrib.layers.fully_connected(self.state, hidden_1_size, activation_fn=None)
        self.fc_1_bn = tf.contrib.layers.batch_norm(self.fc_1, center=True, scale=True, is_training=self.phase)
        self.fc_1_ac = tf.maximum(self.fc_1_bn, self.fc_1_bn*0.01)
        self.fc_2 = tf.contrib.layers.fully_connected(self.fc_1_ac, hidden_2_size, activation_fn=None)
        self.fc_2_bn = tf.contrib.layers.batch_norm(self.fc_2, center=True, scale=True, is_training=self.phase)
        self.fc_2_ac = tf.maximum(self.fc_2_bn, self.fc_2_bn*0.01)
        
        # advantage and value streams
        self.streamA,self.streamV = tf.split(self.fc_2_ac,2,axis=1)
        self.AW = tf.Variable(tf.random_normal([hidden_2_size//2,self.num_actions]))
        self.VW = tf.Variable(tf.random_normal([hidden_2_size//2,1]))
        self.Advantage = tf.matmul(self.streamA,self.AW)
        self.Value = tf.matmul(self.streamV,self.VW)
        
        #Then combine them together to get our final Q-values.
        self.q_output = self.Value + tf.subtract(self.Advantage,tf.reduce_mean(self.Advantage,axis=1,keep_dims=True))

        #Below we obtain the loss by taking the sum of squares difference between the target and predicted Q values.
        self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions,self.num_actions,dtype=tf.float32)
        
        # Importance sampling weights for PER, used in network update         
        self.imp_weights = tf.placeholder(shape=[None], dtype=tf.float32)
        
        # select the Q values for the actions that would be selected         
        self.Q = tf.reduce_sum(tf.multiply(self.q_output, self.actions_onehot), reduction_indices=1) # batch size x 1 vector
        
        # regularisation penalises the network when it produces rewards that are above the
        # reward threshold, to ensure reasonable Q-value predictions      
        self.reg_vector = tf.maximum(tf.abs(self.Q)-REWARD_THRESHOLD,0)
        self.reg_term = tf.reduce_sum(self.reg_vector)
        
        self.abs_error = tf.abs(self.targetQ - self.Q)
        
        self.td_error = tf.square(self.targetQ - self.Q)
        
        # below is the loss when we are not using PER
        self.old_loss = tf.reduce_mean(self.td_error)
        
        # as in the paper, to get PER loss we weight the squared error by the importance weights
        self.per_error = tf.multiply(self.td_error, self.imp_weights)

        # total loss is a sum of PER loss and the regularisation term
        if per_flag:
            self.loss = tf.reduce_mean(self.per_error) + reg_lambda*self.reg_term
        else:
            self.loss = self.old_loss + reg_lambda*self.reg_term

        self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self.update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(self.update_ops):
        # Ensures that we execute the update_ops before performing the model update, so batchnorm works
            self.update_model = self.trainer.minimize(self.loss)

In [42]:
def update_target_graph(tf_vars,tau):
    total_vars = len(tf_vars)
    op_holder = []
    for idx,var in enumerate(tf_vars[0:int(total_vars/2)]):
        op_holder.append(tf_vars[idx+int(total_vars/2)].assign((var.value()*tau) + ((1-tau)*tf_vars[idx+int(total_vars/2)].value())))
    return op_holder

In [43]:
def update_target(op_holder,sess):
    for op in op_holder:
        sess.run(op)

In [44]:
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [45]:
def process_train_batch(size):
    if per_flag:
        # uses prioritised exp replay
        a = df.sample(n=size, weights=df['prob'])
    else:
        a = df.sample(n=size)
    states = None
    actions = None
    rewards = None
    next_states = None
    next_actions = None
    done_flags = None
    for i in a.index:
        cur_state = a.loc[i,state_features]
        iv = int(a.loc[i, 'iv_input'])
        vaso = int(a.loc[i, 'vaso_input'])
        action = action_map[iv,vaso]
        reward = a.loc[i,'reward']

        if i != df.index[-1]:
            # if not terminal step in trajectory             
            if df.loc[i, 'icustayid'] == df.loc[i+1, 'icustayid']:
                next_state = df.loc[i + 1, state_features]
                next_vaso = int(df.loc[i+1, 'vaso_input'])
                next_iv = int(df.loc[i+1, 'iv_input'])
                next_action = action_map[iv,vaso]
                done = 0
            else:
                # trajectory is finished
                next_state = np.zeros(len(cur_state))
                next_action = 0
                done = 1
        else:
            # last entry in df is the final state of that trajectory
            next_state = np.zeros(len(cur_state))
            next_action = 0
            done = 1
    
        if states is None:
            states = copy.deepcopy(cur_state)
        else:
            states = np.vstack((states,cur_state))

        if actions is None:
            actions = [action]
        else:
            actions = np.vstack((actions,action))

        if rewards is None:
            rewards = [reward]
        else:
            rewards = np.vstack((rewards,reward))

        if next_states is None:
            next_states = copy.deepcopy(next_state)
        else:
            next_states = np.vstack((next_states,next_state))
        
        if next_actions is None:
            next_actions = [next_action]
        else:
            next_actions = np.vstack((next_actions,next_action))

        if done_flags is None:
            done_flags = [done]
        else:
            done_flags = np.vstack((done_flags,done))
    
    return (states, np.squeeze(actions), np.squeeze(rewards), next_states,np.squeeze(next_actions), np.squeeze(done_flags), a)

In [46]:
def process_eval_batch(size, eval_type = None):
    if eval_type is None:
        raise Exception('Provide eval_type to process_eval_batch')
    elif eval_type == 'train':
        a = df.copy()
    elif eval_type == 'val':
        a = val_df.copy()
    elif eval_type == 'test':
        a = test_df.copy()
    elif eval_type == 'pregnant':
        a = preg_df.copy()
    elif eval_type == 'control':
        a = control_1_df.copy()
    elif eval_type == 'control_sec':
        a = control_2_df.copy()
    elif eval_type == 'old':
        a = old_df.copy()
    else:
        raise Exception('Unknown eval_type')

    count = 0
    while count < len(a.index):
        states = None
        actions = None
        rewards = None
        next_states = None
        next_actions = None
        done_flags = None

        start_idx = count
        end_idx = min(len(a.index), count+size)
        segment = a.index[start_idx:end_idx]
        
        for i in segment:
            cur_state = a.loc[i,state_features]
            iv = int(a.loc[i, 'iv_input'])
            vaso = int(a.loc[i, 'vaso_input'])
            action = action_map[iv,vaso]
            reward = a.loc[i,'reward']

            if i != a.index[-1]:
                # if not terminal step in trajectory             
                if a.loc[i, 'icustayid'] == a.loc[i+1, 'icustayid']:
                    next_state = a.loc[i + 1, state_features]
                    next_vaso = int(a.loc[i+1, 'vaso_input'])
                    next_iv = int(a.loc[i+1, 'iv_input'])
                    next_action = action_map[iv,vaso]
                    done = 0
                else:
                    # trajectory is finished
                    next_state = np.zeros(len(cur_state))
                    next_action = 0
                    done = 1

            else:
                # last entry in df is the final state of that trajectory
                next_state = np.zeros(len(cur_state))
                done = 1

            if states is None:
                states = copy.deepcopy(cur_state)
            else:
                states = np.vstack((states,cur_state))

            if actions is None:
                actions = [action]
            else:
                actions = np.vstack((actions,action))

            if rewards is None:
                rewards = [reward]
            else:
                rewards = np.vstack((rewards,reward))

            if next_states is None:
                next_states = copy.deepcopy(next_state)
            else:
                next_states = np.vstack((next_states,next_state))

            if next_actions is None:
                next_actions = [next_action]
            else:
                next_actions = np.vstack((next_actions,next_action))

            if done_flags is None:
                done_flags = [done]
            else:
                done_flags = np.vstack((done_flags,done))

        yield (states, np.squeeze(actions), np.squeeze(rewards), next_states,np.squeeze(next_actions), np.squeeze(done_flags), a)
        
        count += size

In [47]:
phys_q_train = []
phys_actions_tr = []
def train_set_performance():
    count = 0
    global phys_q_train
    global phys_actions
    phys_q_train = []
    phys_actions_tr = []
    for r in df.index:
        cur_state = [df.loc[r,state_features]]
        iv = int(df.loc[r, 'iv_input'])
        vaso = int(df.loc[r, 'vaso_input'])
        action = action_map[iv,vaso]
        output_q = np.squeeze(sess.run(mainQN.q_output, feed_dict = {mainQN.state : cur_state, mainQN.phase : False}))
        phys_q_train.append(output_q[action])
        phys_actions_tr.append(action)
        count += 1

In [48]:
def do_eval(eval_type):
    gen = process_eval_batch(size = 1000, eval_type=eval_type)

    phys_q_ret = []
    actions_ret = []
    error_ret = 0

    for b in gen:
        states,actions,rewards,next_states, next_actions, done_flags, _ = b
        
        # Q values for the next timestep from target network, as part of the target Q calculation
        Q2 = sess.run(targetQN.q_output,feed_dict={targetQN.state:next_states, targetQN.phase : 0})

        # handles the case when a trajectory is finished
        end_multiplier = 1 - done_flags

        # Using the next actions, find the q value for the next state/action pairs
        next_state_q = Q2[range(len(Q2)),next_actions]

        # definition of target Q
        targetQ = rewards + (gamma*next_state_q * end_multiplier)

        # get the output q's, actions, and loss
        q_output, abs_error = sess.run([mainQN.q_output, mainQN.abs_error], \
            feed_dict={mainQN.state:states,
                       mainQN.targetQ:targetQ, 
                       mainQN.actions:actions,
                       mainQN.phase:False})
        
        phys_q = q_output[range(len(q_output)), actions]
        error = np.mean(abs_error)
        
        #  update the return vals
        phys_q_ret.extend(phys_q)
        actions_ret.extend(actions)
        error_ret += error

    return phys_q_ret, actions_ret, error_ret

In [49]:
config = tf.ConfigProto()
#config.gpu_options.allow_growth = True  # Don't use all GPUs 
#config.allow_soft_placement = True  # Enable manual control

In [50]:
def do_save_results():
    # get the chosen actions for the train, val, and test set when training is complete.
    phys_q_train, phys_actions_train, _ =  do_eval(eval_type = 'train')        
    phys_q_val, phys_actions_val, _ = do_eval(eval_type = 'val')        
    phys_q_test, phys_actions_test, _ = do_eval(eval_type = 'test')
    #phys_q_preg, phys_actions_preg, _ = do_eval(eval_type = 'pregnant')
    #phys_q_cont, phys_actions_cont, _ = do_eval(eval_type = 'control')
    #phys_q_old, phys_actions_old, _ = do_eval(eval_type = 'old')
    
    # save everything for later - they're used in policy evaluation and when generating plots
    with open(save_dir + 'phys_actions_train.p', 'wb') as f:
        pickle.dump(phys_actions_train, f)
    with open(save_dir + 'phys_actions_val.p', 'wb') as f:
        pickle.dump(phys_actions_val, f)
    with open(save_dir + 'phys_actions_test.p', 'wb') as f:
        pickle.dump(phys_actions_test, f)
    #with open(save_dir + 'phys_actions_preg.p', 'wb') as f:
    #    pickle.dump(phys_actions_preg, f)
    #with open(save_dir + 'phys_actions_cont.p', 'wb') as f:
    #    pickle.dump(phys_actions_cont, f)
    #with open(save_dir + 'phys_actions_old.p', 'wb') as f:
    #    pickle.dump(phys_actions_old, f)
        
    with open(save_dir + 'phys_q_train.p', 'wb') as f:
        pickle.dump(phys_q_train, f)
    with open(save_dir + 'phys_q_val.p', 'wb') as f:
        pickle.dump(phys_q_val, f)
    with open(save_dir + 'phys_q_test.p', 'wb') as f:
        pickle.dump(phys_q_test, f)
    #with open(save_dir + 'phys_q_preg.p', 'wb') as f:
    #    pickle.dump(phys_q_preg, f)
    #with open(save_dir + 'phys_q_cont.p', 'wb') as f:
    #    pickle.dump(phys_q_cont, f)
    #with open(save_dir + 'phys_q_old.p', 'wb') as f:
    #    pickle.dump(phys_q_old, f)
    return

In [51]:
def do_final_save_results():
    # get the chosen actions for the train, val, and test set when training is complete.
    _, _, agent_q_train, agent_actions_train, _ = do_eval(eval_type = 'train')
    print ("length IS ", len(agent_actions_train))
    with open(save_dir + 'phys_actions_train.p', 'wb') as f:
        pickle.dump(agent_actions_train, f)
    _, _, agent_q_val, agent_actions_val, _ = do_eval(eval_type = 'val')        
    _, _, agent_q_test, agent_actions_test, _ = do_eval(eval_type = 'test')   
    _, _, agent_q_preg, agent_actions_preg, _ = do_eval(eval_type = 'pregnant')
    _, _, agent_q_control, agent_actions_control, _ = do_eval(eval_type = 'control')
    _, _, agent_q_old, agent_actions_old, _ = do_eval(eval_type = 'old')
    _, _, agent_q_control_sec, agent_actions_control_sec, _ = do_eval(eval_type = 'control_sec')
    
    # save everything for later - they're used in policy evaluation and when generating plots
    with open(save_dir + 'phys_actions_train.p', 'wb') as f:
        pickle.dump(agent_actions_train, f)
    with open(save_dir + 'phys_actions_val.p', 'wb') as f:
        pickle.dump(agent_actions_val, f)
    with open(save_dir + 'phys_actions_test.p', 'wb') as f:
        pickle.dump(agent_actions_test, f)
    with open(save_dir + 'phys_actions_preg.p', 'wb') as f:
        pickle.dump(agent_actions_preg, f)
    with open(save_dir + 'phys_actions_cont.p', 'wb') as f:
        pickle.dump(agent_actions_control, f)
    with open(save_dir + 'phys_actions_old.p', 'wb') as f:
        pickle.dump(agent_actions_old, f)
    with open(save_dir + 'phys_actions_cont_sec.p', 'wb') as f:
        pickle.dump(agent_actions_control_sec, f)
        
    with open(save_dir + 'phys_q_train.p', 'wb') as f:
        pickle.dump(agent_q_train, f)
    with open(save_dir + 'phys_q_val.p', 'wb') as f:
        pickle.dump(agent_q_val, f)
    with open(save_dir + 'phys_q_test.p', 'wb') as f:
        pickle.dump(agent_q_test, f)
    with open(save_dir + 'phys_q_preg.p', 'wb') as f:
        pickle.dump(agent_q_preg, f)
    with open(save_dir + 'phys_q_cont.p', 'wb') as f:
        pickle.dump(agent_q_control, f)
    with open(save_dir + 'phys_q_old.p', 'wb') as f:
        pickle.dump(agent_q_old, f)
    with open(save_dir + 'phys_q_cont-sec.p', 'wb') as f:
        pickle.dump(agent_q_control_sec, f)
    return

In [42]:
#  # The main training loop is here
# per_alpha = 0.6 # PER hyperparameter
# per_epsilon = 0.01 # PER hyperparameter
# batch_size = 32
# gamma = 0.99 # discount factor 
# num_steps = 40000 # How many steps to train for
# load_model = True #Whether to load a saved model.
# save_dir = "../Created Data/sarsa_phys_2/"
# save_path = "../Created Data/sarsa_phys_2/ckpt"#The path to save our model to.
# tau = 0.001 #Rate to update target network toward primary network
# tf.reset_default_graph()
# mainQN = Qnetwork()
# targetQN = Qnetwork()
# av_q_list = []
# save_results = False

# saver = tf.train.Saver(tf.global_variables())

# init = tf.global_variables_initializer()

# trainables = tf.trainable_variables()

# target_ops = update_target_graph(trainables,tau)

# #Make a path for our model to be saved in.
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

# with tf.Session(config=config) as sess:
#     if load_model == True:
#         print('Trying to load model...')
#         try:
#             restorer = tf.train.import_meta_graph(save_path + '.meta')
#             restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
#             print ("Model restored")
#         except IOError:
#             print ("No previous model found, running default init")
#             sess.run(init)
#         try:
#             per_weights = pickle.load(open( save_dir + "per_weights.p", "rb" ))
#             imp_weights = pickle.load(open( save_dir + "imp_weights.p", "rb" ))
          
#             # the PER weights, governing probability of sampling, and importance sampling
#             # weights for use in the gradient descent updates
#             df['prob'] = per_weights
#             df['imp_weight'] = imp_weights
#             print ("PER and Importance weights restored")
#         except IOError:
#             print("No PER weights found - default being used for PER and importance sampling")
#     else:
#         print("Running default init")
#         sess.run(init)
#     print("Init done")
#     net_loss = 0.0
#     for i in range(num_steps):
#         if save_results:
#             do_save_results()
#             break

#         states,actions,rewards,next_states, next_actions, done_flags, sampled_df = process_train_batch(batch_size)
      
#         # Q values for the next timestep from target network, as part of the update step
#         Q2 = sess.run(targetQN.q_output,feed_dict={targetQN.state:next_states, targetQN.phase : 1})
#          # handles the case when a trajectory is finished
#         end_multiplier = 1 - done_flags
#          # Using the next actions, find the q value for the next state/action pairs
#         next_state_q = Q2[range(len(Q2)),next_actions]
        
# #         # empirical hack to make the Q values never exceed the threshold - helps learning
#         next_state_q[next_state_q > REWARD_THRESHOLD] = REWARD_THRESHOLD
#         next_state_q[next_state_q < -REWARD_THRESHOLD] = -REWARD_THRESHOLD
        
        
# #         # definition of target Q
#         targetQ = rewards + (gamma*next_state_q * end_multiplier)

# #         # Calculate the importance sampling weights for PER
#         imp_sampling_weights = np.array(sampled_df['imp_weight'] / float(max(df['imp_weight'])))
#         imp_sampling_weights[np.isnan(imp_sampling_weights)] = 1
#         imp_sampling_weights[imp_sampling_weights <= 0.001] = 0.001

# #         # Train with the batch
#         _,loss, error, q_tmp = sess.run([mainQN.update_model,mainQN.loss, mainQN.abs_error, mainQN.Q], \
#             feed_dict={mainQN.state:states,
#                        mainQN.targetQ:targetQ, 
#                        mainQN.actions:actions,
#                        mainQN.phase:True,
#                        mainQN.imp_weights:imp_sampling_weights})

# #         # Update target towards main network
#         update_target(target_ops,sess)

#         net_loss += sum(error)
        
# #         # Set the selection weight/prob to the abs prediction error and update the importance sampling weight
#         new_weights = pow((error + per_epsilon), per_alpha)
#         df.loc[df.index.isin(sampled_df.index), 'prob'] = new_weights
#         temp = 1.0/new_weights
#         df.loc[df.index.isin(sampled_df.index), 'imp_weight'] = pow(((1.0/len(df)) * temp), beta_start)
        

#         if i % 1000 == 0 and i > 0:
#             saver.save(sess,save_path)
#             print("Saved Model, step is " + str(i))
           
#             av_loss = net_loss/(1000.0 * batch_size)
#             print("Average loss is ", av_loss)
#             net_loss = 0.0
                        
#             print ("Saving PER and importance weights")
#             with open(save_dir + 'per_weights.p', 'wb') as f:
#                 pickle.dump(df['prob'], f)
#             with open(save_dir + 'imp_weights.p', 'wb') as f:
#                 pickle.dump(df['imp_weight'], f)


#         if (i % 5000==0) and i > 0:
#              # run an evaluation on the validation set
#             phys_q,_, mean_abs_error = do_eval(eval_type = 'val')        
#             print (np.mean(phys_q))
#             print (mean_abs_error)
#             if (i % 30000==0) and i > 0:
#                 print ("Saving results")
#                 do_save_results()
#  #                 break
#     do_final_save_results()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Trying to load model...
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from D:/MIMICIII/Created Data/sarsa_phys_2/ckpt
Model restored
PER and Importance weights restored
Init done
Saved Model, step is 1000
Average loss is  2.1928872367876755
Saving PER and importance weights
Saved Model, step is 2000
Average loss is  1.989621184968215
Saving PER and importance weights
Saved Model, step is 3000
Average loss is  1.9782968829642924
Saving PER and importance weight

ValueError: not enough values to unpack (expected 5, got 3)

In [54]:
gamma = 0.99 # discount factor 
num_steps = 100000 # How many steps to train for
load_model = True #Whether to load a saved model.
save_dir = "D:/MIMICIII/Created Data/sarsa_phys/"
save_path = "D:/MIMICIII/Created Data/sarsa_phys/ckpt"#The path to save our model to.
tau = 0.001 #Rate to update target network toward primary network
tf.reset_default_graph()
mainQN = Qnetwork()
targetQN = Qnetwork()
av_q_list = []
saver = tf.train.Saver(tf.global_variables())

init = tf.global_variables_initializer()

trainables = tf.trainable_variables()

target_ops = update_target_graph(trainables,tau)

#Make a path for our model to be saved in.
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

with tf.Session(config=config) as sess:
    if load_model == True:
        print('Trying to load model...')
        try:
            restorer = tf.train.import_meta_graph(save_path + '.meta')
            restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
            print ("Model restored")
        except IOError:
            print ("No previous model found, running default init")
            sess.run(init)
        try:
            per_weights = pickle.load(open( save_dir + "per_weights.p", "rb" ))
            imp_weights = pickle.load(open( save_dir + "imp_weights.p", "rb" ))
            
            # the PER weights, governing probability of sampling, and importance sampling
            # weights for use in the gradient descent updates
            df['prob'] = per_weights
            df['imp_weight'] = imp_weights
            print ("PER and Importance weights restored")
        except IOError:
            print("No PER weights found - default being used for PER and importance sampling")
    else:
        print("Running default init")
        sess.run(init)
    print("Init done")
    
    do_save_results()
    
    phys_q_old, phys_actions_old, _ = do_eval(eval_type = 'old')
    with open(save_dir + 'phys_actions_old.p', 'wb') as f:
        pickle.dump(phys_actions_old, f)
    with open(save_dir + 'phys_q_old.p', 'wb') as f:
        pickle.dump(phys_q_old, f)   
        
    phys_q_preg, phys_actions_preg, _ = do_eval(eval_type = 'pregnant')
    with open(save_dir + 'phys_actions_preg.p', 'wb') as f:
        pickle.dump(phys_actions_preg, f)
    with open(save_dir + 'phys_q_preg.p', 'wb') as f:
        pickle.dump(phys_q_preg, f) 
    
    phys_q_cont, phys_actions_cont, _ = do_eval(eval_type = 'control')
    with open(save_dir + 'phys_actions_cont.p', 'wb') as f:
        pickle.dump(phys_actions_cont, f)
    with open(save_dir + 'phys_q_cont.p', 'wb') as f:
        pickle.dump(phys_q_cont, f) 
    
    phys_q_cont_sec, phys_actions_cont_sec, _ = do_eval(eval_type = 'control_sec')
    with open(save_dir + 'phys_actions_cont_sec.p', 'wb') as f:
        pickle.dump(phys_actions_cont_sec, f)
    with open(save_dir + 'phys_q_cont_sec.p', 'wb') as f:
        pickle.dump(phys_q_cont_sec, f)
    

Trying to load model...
INFO:tensorflow:Restoring parameters from D:/MIMICIII/Created Data/sarsa_phys/ckpt
Model restored
PER and Importance weights restored
Init done


In [55]:
8

8